## test hierarchical merge in FOF algorithm

#### the idea is this: 

* after the local FOF stage, each partition reports the particles it holds in the overlap region
* do a reduceByKey or treeAggregate of some sort to collect the groups belonging to the same particles
* produce a mapping of $G -> G_1$ and distribute to all hosts in form of broadcast lookup table

In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.setrecursionlimit(sys.getrecursionlimit()*10)
import matplotlib.patches as patches

plt.style.use('bmh')

/Users/rok/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import spark_fof
import spark_fof_c
from fof import fof
%load_ext Cython

In [3]:
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))

## Set up data

In [89]:
# create the arrays
from spark_fof_c import pdt
pdt_tipsy = np.dtype([('mass', 'f4'),('pos', 'f4', 3),('vel', 'f4', 3), ('eps', 'f4'), ('phi', 'f4')])
# nps = 1000000
# ngs = 1
# particles = np.zeros(nps, dtype=pdt)
# done_ps = 0
# #centers = np.random.rand(ngs,3)*1.7 - 0.85
# centers = np.array([0,0,0]).reshape(1,3)
# for group, center in zip(range(ngs), centers): 
#     print group, center
#     group_ps = nps/ngs
#     if nps - (done_ps + group_ps) < group_ps:
#         group_ps = nps - done_ps 
#     particles['pos'][done_ps:done_ps+group_ps] = \
#         np.random.multivariate_normal(center, [[.5,0,0],[0,.5,0],[0,0,.5]], group_ps)
#     done_ps += group_ps
   
# particles['iOrder'] = range(nps)

In [90]:
pdt


dtype({'names':['pos','iGroup','iOrder'], 'formats':[('<f4', (3,)),'<i8','<i4'], 'offsets':[0,16,24], 'itemsize':32}, align=True)

<function newbyteorder>

In [96]:
pdt_noalign = np.dtype([('pos', 'f4', 3), ('iGroup', 'i8'), ('iOrder', 'i4')])

In [5]:
from spark_fof_c import pdt

## Start Spark

In [6]:
import findspark
findspark.init()

In [7]:
import os
os.environ['SPARK_CONF_DIR'] = './conf'
os.environ['SPARK_DRIVER_MEMORY'] = '4G'

In [8]:
import pyspark
from pyspark import SparkContext, SparkConf
import pynbody

In [9]:
conf = SparkConf()

In [10]:
conf.set('spark.python.profile', 'true')
conf.set('spark.executor.memory', '3G')
conf.set('spark.driver.memory', '4G')

In [11]:
sc = SparkContext(master='local[*]', conf=conf)

In [12]:
sc.addPyFile('spark_fof.py')
sc.addPyFile('spark_util.py')
sc.addPyFile('spark_fof_c.pyx')
sc.addPyFile('spark_fof_c.c')
sc.addPyFile('spark_fof_c.so')
sc.addPyFile('fof.so')

## Set up the domains

In [13]:
N = 1
tau = 7.8125e-4
mins = np.array([-.5,-.5,-.5], dtype=np.float)
maxs= np.array([.5,.5,.5], dtype=np.float)
domain_containers = spark_fof.setup_domain(N,tau,maxs,mins)

In [14]:
# f, ax = plt.subplots(subplot_kw={'aspect':'equal'}, figsize=(15,15))
# pynbody.plot.image(s.d, width=1, units = 'Msol Mpc^-2', cmap=plt.cm.Greys, show_cbar=False, subplot=ax)
# #plot_rectangle(domain_containers[0].bufferRectangle, ax=ax)
# for p in particles[::1000000]: 
#     plot_rectangle(domain_containers[spark_fof.get_bin_cython(p['pos'],2**N, np.array(mins), np.array(maxs))], ax=ax)
#     plot_rectangle(domain_containers[spark_fof.get_bin_cython(p['pos'], 2**N, np.array(mins),np.array(maxs))].bufferRectangle, ax=ax)
#     ax.plot(p['pos'][0], p['pos'][1], '.')
# plt.draw()
# ax.set_xlim(-.5,.5)
# ax.set_ylim(-.5,.5)

### Make the base RDD

In [29]:
from pyspark.accumulators import AccumulatorParam

class dictAdd(AccumulatorParam):
    def zero(self, value):
        return {i:0 for i in range(len(value))}
    def addInPlace(self, val1, val2): 
        for k, v in val2.iteritems(): 
            val1[k] += v
        return val1
    
def read_tipsy_output(filename, chunksize = 2048): 
    """
    Read a tipsy file and set the sequential particle IDs
    
    This scans through the data twice -- first to get partition particle counts
    and a second time to actually set the particle IDs.
    """
    
    # helper functions
    def convert_to_fof_particle(s): 
        p_arr = np.frombuffer(s, pdt_tipsy)

        new_arr = np.zeros(len(p_arr), dtype=pdt)
        new_arr['pos'] = p_arr['pos']    
        return new_arr

    def convert_to_fof_particle_partition(index, iterator): 
        for s in iterator: 
            a = convert_to_fof_particle(s)
            if count: 
                npart_acc.add({index: len(a)})
            yield a

    def set_particle_IDs_partition(index, iterator): 
        p_counts = partition_counts.value
        local_index = 0
        start_index = sum([p_counts[i] for i in range(index)])
        for arr in iterator:
            arr['iOrder'] = range(start_index + local_index, start_index + local_index + len(arr))
            local_index += len(arr)
            yield arr
    
    rec_rdd = sc.binaryRecords(filename, pdt_tipsy.itemsize*chunksize)
    nPartitions = rec_rdd.getNumPartitions()
    # set the partition count accumulator
    npart_acc = sc.accumulator({i:0 for i in range(nPartitions)}, dictAdd())
    count=True
    # read the data and count the particles per partition
    rec_rdd = rec_rdd.mapPartitionsWithIndex(convert_to_fof_particle_partition)
    rec_rdd.count()
    count=False

    partition_counts = sc.broadcast(npart_acc.value)

    return rec_rdd.mapPartitionsWithIndex(set_particle_IDs_partition)

In [158]:
p_rdd = read_tipsy_output('/Users/rok/polybox/euclid256.nat_no_header', chunksize=1024)

In [42]:
%timeit p_rdd.count()

1 loop, best of 3: 504 ms per loop


In [43]:
len(p_rdd.first())

1024

In [44]:
ps = np.concatenate(p_rdd.collect())

In [45]:
assert(len(ps) == len(pynbody.load('/Users/rok/polybox/euclid256.nat')))

In [46]:
%%time 
nMinMembers = 1
n_groups = fof.run(ps, tau, nMinMembers)
print 'number of groups to %d particle = %d'%(nMinMembers, n_groups)

number of groups to 1 particle = 7251094
CPU times: user 18.2 s, sys: 82.9 ms, total: 18.3 s
Wall time: 18.3 s


### Partition particles into domains and set the partition part of local group ID

In [159]:
# partitioning duplicates the particles that are located in the boundary regions
part_rdd = (p_rdd.mapPartitions(lambda particles: spark_fof.partition_particles_cython(particles, domain_containers, tau, mins, maxs))
                 .partitionBy(len(domain_containers))
                 .values())

In [160]:
part_rdd.count()

16819349

In [161]:
sc.show_profiles()

Profile of RDD<id=66>
         295488 function calls (295452 primitive calls) in 9.055 seconds

   Ordered by: internal time, cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    32786    7.981    0.000    7.981    0.000 {method 'read' of 'file' objects}
    16384    0.384    0.000    0.601    0.000 <ipython-input-29-81e75887d068>:20(convert_to_fof_particle)
    16384    0.157    0.000    0.157    0.000 {numpy.core.multiarray.zeros}
    16402    0.113    0.000    8.984    0.001 <ipython-input-29-81e75887d068>:27(convert_to_fof_particle_partition)
    16402    0.105    0.000    8.155    0.000 serializers.py:155(_read_with_length)
    16384    0.057    0.000    0.057    0.000 {numpy.core.multiarray.frombuffer}
    16402    0.041    0.000    0.089    0.000 serializers.py:542(read_int)
    16384    0.039    0.000    0.047    0.000 <ipython-input-29-81e75887d068>:6(addInPlace)
    16384    0.036    0.000    0.082    0.000 accumulators.py:160(add)
    

In [127]:
x = p_rdd._jrdd.glom().collect()

In [134]:
y = part_rdd._jrdd.glom().collect()

In [186]:
len(y[7])

988

In [174]:
from cPickle import loads, dumps

In [144]:
timeit np.concatenate(loads(x[0][100]))

10 loops, best of 3: 97.8 ms per loop


In [172]:
b = loads(x[0][100])

In [175]:
timeit dumps(b)

100 loops, best of 3: 4.68 ms per loop


In [145]:
timeit loads(y[0][100])

10 loops, best of 3: 143 ms per loop


In [152]:
a = np.concatenate(loads(x[0][100]))
a_str = a.tobytes()

In [153]:
timeit np.frombuffer(a_str, pdt)

The slowest run took 19.30 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 321 ns per loop


In [155]:
321e-9*20

6.4199999999999995e-06

In [156]:
100e-3

0.1

In [154]:
np.frombuffer(a_str,pdt)

array([ ([-0.39915788173675537, -0.26693859696388245, -0.32561394572257996], 0, 203776),
       ([-0.4002014100551605, -0.2683628499507904, -0.32715079188346863], 0, 203777),
       ([-0.38977810740470886, -0.2641215920448303, -0.3301052153110504], 0, 203778),
       ...,
       ([-0.39438652992248535, -0.27104148268699646, -0.30935826897621155], 0, 205821),
       ([-0.3946029543876648, -0.2695358395576477, -0.309119313955307], 0, 205822),
       ([-0.3961787819862366, -0.27145761251449585, -0.30734795331954956], 0, 205823)], 
      dtype={'names':['pos','iGroup','iOrder'], 'formats':[('<f4', (3,)),'<i8','<i4'], 'offsets':[0,16,24], 'itemsize':32, 'aligned':True})

In [138]:
len(loads(x[0][100])[0])

1024

In [139]:
loads(x[0][100])

[array([ ([-0.39915788173675537, -0.26693859696388245, -0.32561394572257996], 0, 203776),
        ([-0.4002014100551605, -0.2683628499507904, -0.32715079188346863], 0, 203777),
        ([-0.38977810740470886, -0.2641215920448303, -0.3301052153110504], 0, 203778),
        ...,
        ([-0.3937629461288452, -0.2793988585472107, -0.3096083700656891], 0, 204797),
        ([-0.3947446346282959, -0.27967506647109985, -0.3088250756263733], 0, 204798),
        ([-0.39515626430511475, -0.27921053767204285, -0.30743318796157837], 0, 204799)], 
       dtype={'names':['pos','iGroup','iOrder'], 'formats':[('<f4', (3,)),'<i8','<i4'], 'offsets':[0,16,24], 'itemsize':32, 'aligned':True}),
 array([ ([-0.3962317109107971, -0.27991417050361633, -0.30806562304496765], 0, 204800),
        ([-0.39505261182785034, -0.2805139124393463, -0.3073357045650482], 0, 204801),
        ([-0.39332547783851624, -0.2796914279460907, -0.309701144695282], 0, 204802),
        ...,
        ([-0.39438652992248535, -0.2710414

In [143]:
loads(y[1][100])

[([0.011192538775503635, -0.20581234991550446, -0.2265060693025589], 0, 8585643),
 ([0.01221227366477251, -0.20647509396076202, -0.23139144480228424], 0, 8585644),
 ([0.004160613287240267, -0.20185111463069916, -0.2340831607580185], 0, 8585645),
 ([0.016891494393348694, -0.2100699096918106, -0.24276645481586456], 0, 8585646),
 ([0.023967694491147995, -0.2107529491186142, -0.23905490338802338], 0, 8585647),
 ([0.013937791809439659, -0.2133299708366394, -0.2401094138622284], 0, 8585648),
 ([0.018294252455234528, -0.20785769820213318, -0.24001404643058777], 0, 8585649),
 ([0.01937362551689148, -0.20636248588562012, -0.24167199432849884], 0, 8585650),
 ([0.022161858156323433, -0.2064046859741211, -0.2427632063627243], 0, 8585651),
 ([0.01951485313475132, -0.20537813007831573, -0.23483945429325104], 0, 8585652),
 ([0.01845942623913288, -0.2053600549697876, -0.23991139233112335], 0, 8585653),
 ([0.02166909910738468, -0.2054964303970337, -0.2383510023355484], 0, 8585654),
 ([0.015005717054009

In [131]:
for i in x[1]:
    print len(loads(i))

1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


KeyboardInterrupt: 

In [114]:
len(x[0])

953

In [50]:
sc.show_profiles()

Profile of RDD<id=52>
         33646844 function calls (33646828 primitive calls) in 46.926 seconds

   Ordered by: internal time, cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      874   24.412    0.028   24.412    0.028 {cPickle.loads}
 16819357   13.843    0.000   43.895    0.000 rdd.py:1004(<genexpr>)
 16819349    4.105    0.000    4.105    0.000 rdd.py:1540(<lambda>)
       16    3.031    0.189   46.926    2.933 {sum}
     1756    1.383    0.001    1.383    0.001 {method 'read' of 'file' objects}
      882    0.116    0.000   25.948    0.029 serializers.py:136(load_stream)
      882    0.014    0.000   25.831    0.029 serializers.py:155(_read_with_length)
      874    0.014    0.000   24.425    0.028 serializers.py:421(loads)
      882    0.005    0.000    0.509    0.001 serializers.py:542(read_int)
      882    0.003    0.000    0.003    0.000 {_struct.unpack}
      890    0.001    0.000    0.001    0.000 {len}
        8    0.000    0.0

### Run the local FOF

In [24]:
from spark_util import spark_cython

In [25]:
from fof import fof

In [26]:
def run_local_fof(index, particle_iter, tau, nMinMembers): 
    part_arr = np.fromiter(particle_iter, pdt)
    if len(part_arr)>0:
        fof.run(part_arr, tau, nMinMembers)
    return part_arr

def encode_gid(pid, cid, bits=32):
    if bits == 32: 
        res = np.int64(int(np.binary_repr(pid,width=32)+np.binary_repr(cid,width=32),2))
    elif bits == 16:
        res = np.int32(int(np.binary_repr(pid,width=16)+np.binary_repr(cid,width=16),2))
    else: 
        raise RuntimeError('Group encoding must use either 16 or 32 bit integers')
    return res

def set_group_id(partition_index, particle_iter):
    part_arr = np.fromiter(particle_iter, pdt)
    for i in range(len(part_arr)):
        gid = part_arr['iGroup'][i]
        part_arr['iGroup'][i] = np.int64(bin((partition_index<<32) | gid), 2)
    return part_arr

In [27]:
fof_rdd = part_rdd.mapPartitionsWithIndex(lambda index, particles: run_local_fof(index, particles, tau, 1))\
                  .mapPartitionsWithIndex(set_group_id).cache()

### Group Merging stage

In [28]:
fof_analyzer = spark_fof.FOFAnalyzer(sc, N, tau, fof_rdd, [-.5,-.5,-.5], [.5,.5,.5])

merged_rdd = fof_analyzer.merge_groups(0)

merged = merged_rdd.collect()

merged_arr = np.fromiter(merged, pdt)

groups = np.unique(merged_arr['iGroup'])

In [29]:
assert(len(groups) == n_groups)

In [30]:
len(groups)

7251094

In [31]:
n_groups

7251094